In [ ]:
# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change current working directory
import os
os.chdir('/content/drive/MyDrive/AI/5-B-AI/')

In [ ]:
# check we can see the dataset
os.path.isfile('sepal.csv')

In [ ]:
# work with a Logistic Regression classifier

# data preparation steps

# Importing the packages we use
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay
import seaborn as sns

# Loading all the observations
observations = pd.read_csv('moons.csv')

# Shuffling all the observations
observations_shuffled = observations.sample(frac=1, random_state=99)

# Setting the fraction of observations we will use for testing
testing_fraction = 0.25
split_index = int(observations_shuffled.shape[0] * testing_fraction)

# Splitting into testing observations and training observations ("horizontal split")
observations_test = observations_shuffled.iloc[:split_index]
observations_train = observations_shuffled.iloc[split_index:]

# Splitting into testing examples and testing labels ("vertical split")
test_examples = observations_test.drop(columns='Label').to_numpy()
test_labels = observations_test['Label'].to_numpy()

# Splitting into training examples and training labels ("vertical split")
train_examples = observations_train.drop(columns='Label').to_numpy()
train_labels = observations_train['Label'].to_numpy()

# model training and model evaluation steps

# Create a Logistic Regression model object (the extra hyperparameter asks to use the "cross entropy" cost function...
# (equivalent to the "log loss" where there are just two classes) we saw in the lecture, during gradient descent)
model = LogisticRegression(multi_class='multinomial', random_state=99)

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(train_examples, train_labels)

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)

# Find the total number of model predictions that matched with the corresponding testing labels
correct_predictions = sum(predictions == test_labels)
# Calculate the model's accuracy: the fraction of predictions that were correct
accuracy = correct_predictions / len(test_labels)
# Display the accuracy as a single quantitative measure of overall performance
print("Accuracy:", accuracy, "(or", round(accuracy*100, 1), "%)")

# visualise the final generalisation (note the small change here...)
plt.figure()
disp = DecisionBoundaryDisplay.from_estimator(model, train_examples, response_method='predict')
sns.scatterplot(x=train_examples[:, 0], y=train_examples[:, 1], hue=train_labels)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Logistic regressor: final generalisation')
plt.show()

In [ ]:
# work with a Logistic Regression classifier (extra features)

# data preparation steps

# Importing the packages we use
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

# Loading all the observations
observations = pd.read_csv('moons.csv')

# Shuffling all the observations
observations_shuffled = observations.sample(frac=1, random_state=99)

# Setting the fraction of observations we will use for testing
testing_fraction = 0.25
split_index = int(observations_shuffled.shape[0] * testing_fraction)

# Splitting into testing observations and training observations ("horizontal split")
observations_test = observations_shuffled.iloc[:split_index]
observations_train = observations_shuffled.iloc[split_index:]

# Splitting into testing examples and testing labels ("vertical split")
test_examples = observations_test.drop(columns='Label').to_numpy()
test_labels = observations_test['Label'].to_numpy()

# Splitting into training examples and training labels ("vertical split")
train_examples = observations_train.drop(columns='Label').to_numpy()
train_labels = observations_train['Label'].to_numpy()

# generate extra predictive features using simple nonlinear transformations (multiplying existing predictive feature 
# values by themselves and each other), for both our training data and our testing data
poly = PolynomialFeatures(2)
poly_train_examples = poly.fit_transform(train_examples)
poly_test_examples = poly.transform(test_examples)

print("we've gone from", train_examples.shape[1], "predictive features, to", poly_train_examples.shape[1])

# model training and model evaluation steps

# Create a Logistic Regression model object (the extra hyperparameter asks to use the "cross entropy" cost function...
# (equivalent to the "log loss" where there are just two classes) we saw in the lecture, during gradient descent)
model = LogisticRegression(multi_class='multinomial', random_state=99)

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(poly_train_examples, train_labels)

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(poly_test_examples)

# Find the total number of model predictions that matched with the corresponding testing labels
correct_predictions = sum(predictions == test_labels)
# Calculate the model's accuracy: the fraction of predictions that were correct
accuracy = correct_predictions / len(test_labels)
# Display the accuracy as a single quantitative measure of overall performance
print("Accuracy:", accuracy, "(or", round(accuracy*100, 1), "%)")

# visualise the final generalisation 

# (this is not something you'd ever need to do outside of a learning-about-ML context...
# so don't worry about the code (which is very similar to what we did for regression visualisation), but some comments
# follow if you're interested: 
# this is more involved now, a helper function like DecisionBoundaryDisplay
# can't cope with >2 predictive featuers, so we'll have to do the visualisation ourselves and manually strip out all the 
# extra features before plotting; we do something very close to what we did for regression visualisation but this is a 
# classification problem, and we show the probability scores for the positive class (the one that comes first 
# alphabetically, by default - so "Class A"); (annoyingly contourf() won't accept string-like class labels, so we can't 
# (easily) show the boundary by plotting with two discrete colour codes across the grid, based on predicted labels 
# (at least not easily without a lot of messing round and recasting)))

# Prepare our own grid of testing examples to use in visualization
x_min = train_examples[:, 0].min() - 1
x_max = train_examples[:, 0].max() + 1
y_min = train_examples[:, 1].min() - 1
y_max = train_examples[:, 1].max() + 1
grid_x, grid_y = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
grid_examples = np.c_[grid_x.ravel(), grid_y.ravel()]

# Apply the same nonlinear transformations to the feature values in our grid
poly_grid_examples = poly.transform(grid_examples)

# Find positive class probabilities for each of the examples in the grid
grid_probabilities = model.predict_proba(poly_grid_examples)[:,0]
grid_probabilities = grid_probabilities.reshape(grid_x.shape)

# Visualise the final generalisation
plt.figure()
contour = plt.contourf(grid_x, grid_y, grid_probabilities, cmap='gray', alpha=0.8)
plt.colorbar(contour, label='Positive class (A) probability')
sns.scatterplot(x=train_examples[:, 0], y=train_examples[:, 1], hue=train_labels, edgecolor='k')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Logistic regression (extra features): final generalisation')
plt.show()

In [ ]:
# work with a Support Vector Machine classifier

# data preparation steps

# Importing the packages we use
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay
import seaborn as sns

# Loading all the observations
observations = pd.read_csv('moons.csv')

# Shuffling all the observations
observations_shuffled = observations.sample(frac=1, random_state=99)

# Setting the fraction of observations we will use for testing
testing_fraction = 0.25
split_index = int(observations_shuffled.shape[0] * testing_fraction)

# Splitting into testing observations and training observations ("horizontal split")
observations_test = observations_shuffled.iloc[:split_index]
observations_train = observations_shuffled.iloc[split_index:]

# Splitting into testing examples and testing labels ("vertical split")
test_examples = observations_test.drop(columns='Label').to_numpy()
test_labels = observations_test['Label'].to_numpy()

# Splitting into training examples and training labels ("vertical split")
train_examples = observations_train.drop(columns='Label').to_numpy()
train_labels = observations_train['Label'].to_numpy()

# model training and model evaluation steps

# Create a Support Vector Machine model object (generates extra predictive features by default)
model = SVC(random_state=99)

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(train_examples, train_labels)

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)

# Find the total number of model predictions that matched with the corresponding testing labels
correct_predictions = sum(predictions == test_labels)
# Calculate the model's accuracy: the fraction of predictions that were correct
accuracy = correct_predictions / len(test_labels)
# Display the accuracy as a single quantitative measure of overall performance
print("Accuracy:", accuracy, "(or", round(accuracy*100, 1), "%)")

# visualise the final generalisation (note the small change here...)
plt.figure()
disp = DecisionBoundaryDisplay.from_estimator(model, train_examples, response_method='predict')
sns.scatterplot(x=train_examples[:, 0], y=train_examples[:, 1], hue=train_labels)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Support Vector Machine: final generalisation')
plt.show()

In [ ]:
# work with a Support Vector Machine regressor

# data preparation steps

# Importing the packages we use
import pandas as pd
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay
import seaborn as sns

# Loading all the observations
observations = pd.read_csv('sepal_regression.csv')

# Shuffling all the observations
observations_shuffled = observations.sample(frac=1, random_state=99)

# Setting the fraction of observations we will use for testing
testing_fraction = 0.25
split_index = int(observations_shuffled.shape[0] * testing_fraction)

# Splitting into testing observations and training observations ("horizontal split")
observations_test = observations_shuffled.iloc[:split_index]
observations_train = observations_shuffled.iloc[split_index:]

# Splitting into testing examples and testing labels ("vertical split")
test_examples = observations_test.drop(columns='petal_length').to_numpy()
test_values = observations_test['petal_length'].to_numpy()

# Splitting into training examples and training labels ("vertical split")
train_examples = observations_train.drop(columns='petal_length').to_numpy()
train_values = observations_train['petal_length'].to_numpy()

# model training and model evaluation steps

# Create a Support Vector Machine model object (generates extra predictive features by default)
model = SVR()

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(train_examples, train_values)

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)

# Find the mean squared error (MSE) between the model's predictions and the testing values
mse = ((predictions - test_values) ** 2).mean()
# Display the MSE as a single quantitative measure of overall performance
print("Mean square error (MSE):", mse)

# Prepare our own grid of testing examples to use in visualization
x_min = train_examples[:, 0].min() - 1
x_max = train_examples[:, 0].max() + 1
y_min = train_examples[:, 1].min() - 1
y_max = train_examples[:, 1].max() + 1
grid_x, grid_y = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
grid_examples = np.c_[grid_x.ravel(), grid_y.ravel()]

# Make predictions for each of the examples in the grid
grid_predictions = model.predict(grid_examples)
grid_predictions = grid_predictions.reshape(grid_x.shape)

# Visualise the final generalisation
plt.figure()
contour = plt.contourf(grid_x, grid_y, grid_predictions, cmap='gray_r', alpha=0.8)
plt.colorbar(contour, label='Petal Length')
sns.scatterplot(x=train_examples[:, 0], y=train_examples[:, 1], hue=train_values, palette='gray_r', edgecolor='k', legend=False)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.title('Support Vector Machine: final generalisation')
plt.show()

In [ ]:
# work with an Artificial Neural Network classifier

# data preparation steps

# Importing the packages we use
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay
import seaborn as sns

# Loading all the observations
observations = pd.read_csv('moons.csv')

# Shuffling all the observations
observations_shuffled = observations.sample(frac=1, random_state=99)

# Setting the fraction of observations we will use for testing
testing_fraction = 0.25
split_index = int(observations_shuffled.shape[0] * testing_fraction)

# Splitting into testing observations and training observations ("horizontal split")
observations_test = observations_shuffled.iloc[:split_index]
observations_train = observations_shuffled.iloc[split_index:]

# Splitting into testing examples and testing labels ("vertical split")
test_examples = observations_test.drop(columns='Label').to_numpy()
test_labels = observations_test['Label'].to_numpy()

# Splitting into training examples and training labels ("vertical split")
train_examples = observations_train.drop(columns='Label').to_numpy()
train_labels = observations_train['Label'].to_numpy()

# model training and model evaluation steps

# Create an Artificial Neural Network model object (using hyperparameters to ask for no hidden layers,
# a logistic activation function, and stochastic gradient descent - equivalent to a logistic regressor)
model = MLPClassifier(hidden_layer_sizes=(), activation='logistic', solver='sgd', random_state=99)

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(train_examples, train_labels)

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)

# Find the total number of model predictions that matched with the corresponding testing labels
correct_predictions = sum(predictions == test_labels)
# Calculate the model's accuracy: the fraction of predictions that were correct
accuracy = correct_predictions / len(test_labels)
# Display the accuracy as a single quantitative measure of overall performance
print("Accuracy:", accuracy, "(or", round(accuracy*100, 1), "%)")

# visualise the final generalisation (note the small change here...)
plt.figure()
disp = DecisionBoundaryDisplay.from_estimator(model, train_examples, response_method='predict')
sns.scatterplot(x=train_examples[:, 0], y=train_examples[:, 1], hue=train_labels)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Artificial Neural Network: final generalisation')
plt.show()

In [ ]:
# work with an Artificial Neural Network classifier

# data preparation steps

# Importing the packages we use
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay
import seaborn as sns

# Loading all the observations
observations = pd.read_csv('moons.csv')

# Shuffling all the observations
observations_shuffled = observations.sample(frac=1, random_state=99)

# Setting the fraction of observations we will use for testing
testing_fraction = 0.25
split_index = int(observations_shuffled.shape[0] * testing_fraction)

# Splitting into testing observations and training observations ("horizontal split")
observations_test = observations_shuffled.iloc[:split_index]
observations_train = observations_shuffled.iloc[split_index:]

# Splitting into testing examples and testing labels ("vertical split")
test_examples = observations_test.drop(columns='Label').to_numpy()
test_labels = observations_test['Label'].to_numpy()

# Splitting into training examples and training labels ("vertical split")
train_examples = observations_train.drop(columns='Label').to_numpy()
train_labels = observations_train['Label'].to_numpy()

# model training and model evaluation steps

# Create an Artificial Neural Network model object (using hyperparameters to ask for one hidden layer
# with 4 nodes, and a good number of epochs for the gradient descent)
model = MLPClassifier(hidden_layer_sizes=(4), max_iter=2000, random_state=99)

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(train_examples, train_labels)

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)

# Find the total number of model predictions that matched with the corresponding testing labels
correct_predictions = sum(predictions == test_labels)
# Calculate the model's accuracy: the fraction of predictions that were correct
accuracy = correct_predictions / len(test_labels)
# Display the accuracy as a single quantitative measure of overall performance
print("Accuracy:", accuracy, "(or", round(accuracy*100, 1), "%)")

# visualise the final generalisation (note the small change here...)
plt.figure()
disp = DecisionBoundaryDisplay.from_estimator(model, train_examples, response_method='predict')
sns.scatterplot(x=train_examples[:, 0], y=train_examples[:, 1], hue=train_labels)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Artificial Neural Network: final generalisation')
plt.show()

In [ ]:
# work with an Artificial Neural Network regressor

# data preparation steps

# Importing the packages we use
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Loading all the observations
observations = pd.read_csv('sepal_regression.csv')

# Shuffling all the observations
observations_shuffled = observations.sample(frac=1, random_state=99)

# Setting the fraction of observations we will use for testing
testing_fraction = 0.25
split_index = int(observations_shuffled.shape[0] * testing_fraction)

# Splitting into testing observations and training observations ("horizontal split")
observations_test = observations_shuffled.iloc[:split_index]
observations_train = observations_shuffled.iloc[split_index:]

# Splitting into testing examples and testing labels ("vertical split")
test_examples = observations_test.drop(columns='petal_length').to_numpy()
test_values = observations_test['petal_length'].to_numpy()

# Splitting into training examples and training labels ("vertical split")
train_examples = observations_train.drop(columns='petal_length').to_numpy()
train_values = observations_train['petal_length'].to_numpy()

# model training and model evaluation steps

# Create an Artificial Neural Network model object (you should be able to interpret the hyperparameters)
model = MLPRegressor(hidden_layer_sizes=(10,10), max_iter=2000, random_state=99)

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(train_examples, train_values)

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)

# Find the mean squared error (MSE) between the model's predictions and the testing values
mse = ((predictions - test_values) ** 2).mean()
# Display the MSE as a single quantitative measure of overall performance
print("Mean square error (MSE):", mse)

# Prepare our own grid of testing examples to use in visualization
x_min = train_examples[:, 0].min() - 1
x_max = train_examples[:, 0].max() + 1
y_min = train_examples[:, 1].min() - 1
y_max = train_examples[:, 1].max() + 1
grid_x, grid_y = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
grid_examples = np.c_[grid_x.ravel(), grid_y.ravel()]

# Make predictions for each of the examples in the grid
grid_predictions = model.predict(grid_examples)
grid_predictions = grid_predictions.reshape(grid_x.shape)

# Visualise the final generalisation
plt.figure()
contour = plt.contourf(grid_x, grid_y, grid_predictions, cmap='gray_r', alpha=0.8)
plt.colorbar(contour, label='Petal Length')
sns.scatterplot(x=train_examples[:, 0], y=train_examples[:, 1], hue=train_values, palette='gray_r', edgecolor='k', legend=False)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.title('Support Vector Machine: final generalisation')
plt.show()

In [ ]:
# work with a simple/small image dataset

# https://en.wikipedia.org/wiki/MNIST_database - handwritten characters

# data preparation steps

# note: normally we would need to load individual image files, and read out and "flatten" the pixel information
# into examples with lots of predictive features (e.g., the MNIST images are of resolution 28*28 pixels, leading 
# to examples with 784 predictive features)... Here, however, just to get the basic ideas quickly, we allow ourselves
# to skip over this step and load up the images from a freely available online file resource, where they have already 
# been "flattened" and associated with their corresponding class labels (the number the person was writing - see the 
# link above for details). We also allow ourselves to use scikit-learn's train_test_split() method for doing the 
# shuffle and split - we've avoided doing this before to ensure we saw and understood the steps for ourselves... But 
# by this point we know the recipe basics and it's fine to make use of the helper...

# Importing the packages we use
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import time

# Loading the MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')

# Examples and labels
examples = mnist['data']
labels = mnist['target']

# Splitting into testing examples and training examples with corresponding labels
train_examples, test_examples, train_labels, test_labels = train_test_split(examples, labels, test_size=0.25, random_state=99)

# model training and model evaluation steps

# Start timing
train_start_time = time.time()

# Create a k-NN model object
model = KNeighborsClassifier()

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(train_examples, train_labels)

# Stop timing
train_end_time = time.time()

# Calculate the elapsed time for training
train_time = train_end_time - train_start_time
print("Training took", train_time, "seconds")

# Start timing for prediction
predict_start_time = time.time()

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)

# Stop timing for prediction
predict_end_time = time.time()

# Calculate the elapsed time for prediction
predict_time = predict_end_time - predict_start_time
print("Prediction took", predict_time, "seconds")

# Find the total number of model predictions that matched with the corresponding testing labels
correct_predictions = sum(predictions == test_labels)
# Calculate the model's accuracy: the fraction of predictions that were correct
accuracy = correct_predictions / len(test_labels)
# Display the accuracy as a single quantitative measure of overall performance
print("Accuracy:", accuracy, "(or", round(accuracy*100, 1), "%)")

# WARNING: though this is a relatively small dataset, containing tiny images...
# ...the code will still take a while to execute (and may take longer if you...
# ...experiment with other models...)